In [ ]:
### 用chatopenai-ollama 替换闭源模型api，发现不会自动调用bind_tools中的工具，可以进行聊天。应该是输出框架不匹配

In [1]:
from typing import Annotated

from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


In [2]:
# import os
# os.environ["LANGCHAIN_TRACING_V2"]="true"
# os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
# os.environ["LANGCHAIN_API_KEY"]="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"
# os.environ["LANGCHAIN_PROJECT"]="pr-sunny-icicle-8"

In [3]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")
_set_env("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")

In [4]:
os.environ["TAVILY_API_KEY"] = "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"
class State(TypedDict):
    messages: Annotated[list, add_messages]
graph_builder = StateGraph(State)


tool = TavilySearchResults(max_results=2)
tools = [tool]

In [5]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

In [6]:
from langchain_anthropic import ChatAnthropic

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="qwen2:7b", api_key="ollama", base_url="http://localhost:11434/v1")
llm_with_tools = llm.bind_tools(tools)
response = llm_with_tools.invoke(["Hello!"])

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


# The first argument is the unique node name
# The second argument is the function or object that will be called whenever
# the node is used.
graph_builder.add_node("chatbot", chatbot)

AttributeError: 'ChatOpenAI' object has no attribute '_create_message_dicts'

In [ ]:
llm_with_tools = llm.bind_tools(tools,tool_choice="auto")
response = llm_with_tools.invoke(["Hello!"])

In [ ]:
print(response)

content='Hello there! How can I assist you today?' response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 11, 'total_tokens': 22}, 'model_name': 'qwen2:7b', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'stop', 'logprobs': None} id='run-630ecf23-f189-4e21-a682-1041defe3710-0' usage_metadata={'input_tokens': 11, 'output_tokens': 11, 'total_tokens': 22}


In [ ]:
import json

from langchain_core.messages import ToolMessage


class BasicToolNode:
    """A node that runs the tools requested in the last AIMessage."""

    def __init__(self, tools: list) -> None:
        self.tools_by_name = {tool.name: tool for tool in tools}

    def __call__(self, inputs: dict):
        if messages := inputs.get("messages", []):
            message = messages[-1]
        else:
            raise ValueError("No message found in input")
        outputs = []
        for tool_call in message.tool_calls:
            tool_result = self.tools_by_name[tool_call["name"]].invoke(
                tool_call["args"]
            )
            outputs.append(
                ToolMessage(
                    content=json.dumps(tool_result),
                    name=tool_call["name"],
                    tool_call_id=tool_call["id"],
                )
            )
        return {"messages": outputs}


tool_node = BasicToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

In [ ]:
from typing import Literal


def route_tools(
    state: State,
) -> Literal["tools", "__end__"]:
    """
    Use in the conditional_edge to route to the ToolNode if the last message
    has tool calls. Otherwise, route to the end.
    """
    if isinstance(state, list):
        ai_message = state[-1]
    elif messages := state.get("messages", []):
        ai_message = messages[-1]
    else:
        raise ValueError(f"No messages found in input state to tool_edge: {state}")
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
        return "tools"
    return "__end__"


# The `tools_condition` function returns "tools" if the chatbot asks to use a tool, and "__end__" if
# it is fine directly responding. This conditional routing defines the main agent loop.
graph_builder.add_conditional_edges(
    "chatbot",
    route_tools,
    # The following dictionary lets you tell the graph to interpret the condition's outputs as a specific node
    # It defaults to the identity function, but if you
    # want to use a node named something else apart from "tools",
    # You can update the value of the dictionary to something else
    # e.g., "tools": "my_tools"
    {"tools": "tools", "__end__": "__end__"},
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
graph = graph_builder.compile()

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    print("out")
    pass

KeyboardInterrupt: 

In [ ]:
for event in graph.stream({"messages": ("user", "what's langgraph all about?")}):
        for value in event.values():
            if isinstance(value["messages"][-1], BaseMessage):
                print("Assistant:", value["messages"][-1].content)

Assistant: LangGraph is a tool designed for natural language processing (NLP) and knowledge graph tasks. It focuses on integrating text-based information with structured data, often representing relationships between entities in the form of a graph database.

Here are some key aspects of LangGraph:

1. **Integration**: LangGraph allows for seamless integration of unstructured textual data with structured datasets by leveraging semantic technologies. This means it can combine knowledge from text (like descriptions or web content) and formalized databases.

2. **Entity Linking**: It enables the process of linking mentions in the text to their corresponding entities in a knowledge base. For example, referring to "Apple Inc." as "Company X" could be recognized by LangGraph and linked back to Apple's actual data record.

3. **Reasoning over Knowledge Graphs**: With LangGraph, you can reason about complex relationships within large knowledge graphs. This is crucial for tasks like answering q